In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings 
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, VotingRegressor

import xgboost as xgb
import lightgbm as lgb

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import optuna

In [3]:
# Data reading 
train_data = pd.read_csv("/kaggle/input/playground-series-s5e4/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s5e4/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s5e4/sample_submission.csv")

In [4]:
# Configuration 
numerical_variables = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']
categorial_valiable = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment'] 
target_variable = 'Listening_Time_minutes'

In [5]:
# Utility function 

def noise_removal(df):
    df["Number_of_Ads"] = df['Number_of_Ads'].apply(lambda x: x if x < 4 else 4)
    df["Episode_Length_minutes"] = df['Episode_Length_minutes'].apply(lambda x: x if x < 150 else 150)
    return df

def missing_value_imputation(df):
    df["Episode_Length_minutes"] = df.groupby("Podcast_Name")["Episode_Length_minutes"].transform(lambda x: x.fillna(x.mean()))
    df["Guest_Popularity_percentage"] = df.groupby("Podcast_Name")["Guest_Popularity_percentage"].transform(lambda x: x.fillna(x.mean()))
    df["Number_of_Ads"] = df["Number_of_Ads"].fillna(2)
    return df

def bin_popularity(value):
    if value < 20:
        return 'Low'
    elif value < 60:
        return 'Medium'
    else:
        return 'High'

INTERACT = []
def feature_engineering(train, test):
    global numerical_variables
    # Numerical columns 
    ## Interaction features
    if len(INTERACT) > 0:
        INTERACT.clear()
    for i,c1 in enumerate(numerical_variables):
        for j,c2 in enumerate(numerical_variables[i+1:]):
            feature = f"{c1}_{c2}"
            train[feature] = train[c1] * train[c2]
            test[feature] = test[c1] * test[c2]
            INTERACT.append(feature)
    numerical_variables += INTERACT
    print(f"There are {len(INTERACT)} interaction features:")
    print(INTERACT)
    
    train['Host_Guest_Avg_Popularity'] = (train['Host_Popularity_percentage'] + train['Guest_Popularity_percentage']) / 2
    test['Host_Guest_Avg_Popularity'] = (test['Host_Popularity_percentage'] + test['Guest_Popularity_percentage']) / 2
    numerical_variables.append("Host_Guest_Avg_Popularity")
    
    train["Host_Popularity_Level"] = train["Host_Popularity_percentage"].apply(bin_popularity)
    test["Host_Popularity_Level"] = test["Host_Popularity_percentage"].apply(bin_popularity)
    categorial_valiable.append("Host_Popularity_Level")
    train["Guest_Popularity_Level"] = train["Guest_Popularity_percentage"].apply(bin_popularity)
    test["Guest_Popularity_Level"] = test["Guest_Popularity_percentage"].apply(bin_popularity)
    categorial_valiable.append("Guest_Popularity_Level")
    
    train['Has_Ads'] = train['Number_of_Ads'].apply(lambda x: 1 if x > 0 else 0)
    test['Has_Ads'] = test['Number_of_Ads'].apply(lambda x: 1 if x > 0 else 0)
    
    train["Ads_Per_Minutes"] = train["Episode_Length_minutes"] / (train["Number_of_Ads"]+1)
    test["Ads_Per_Minutes"] = test["Episode_Length_minutes"] / (test["Number_of_Ads"]+1)
    numerical_variables.append("Ads_Per_Minutes")

    # Categorical features
    train['Is_Weekend'] = train['Publication_Day'].apply(lambda x: 1 if x == "Saturday" or x == "Sunday" else 0)
    test['Is_Weekend'] = test['Publication_Day'].apply(lambda x: 1 if x == "Saturday" or x == "Sunday" else 0)

    return train, test

In [6]:
def one_hot_encode_features(train, test, categorical_features):
    # Initialize the OneHotEncoder with drop='first'
    encoder = OneHotEncoder(sparse=False)
    
    # Fit and transform the train set
    train_encoded = encoder.fit_transform(train[categorical_features])
    
    # Transform the test set
    test_encoded = encoder.transform(test[categorical_features])
    
    # Convert the encoded arrays to DataFrame
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_features))
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_features))
    
    # Drop the original categorical columns from train and test DataFrames
    train.drop(columns=categorical_features, inplace=True)
    test.drop(columns=categorical_features, inplace=True)
    
    # Concatenate the one-hot encoded features to the original DataFrames
    train = pd.concat([train, train_encoded_df], axis=1)
    test = pd.concat([test, test_encoded_df], axis=1)
    
    return train, test


def standardize_features(train, test, numerical_features):
    # Initialize the StandardScaler
    scaler = StandardScaler()
    
    # Fit and transform the training data
    train_scaled = scaler.fit_transform(train[numerical_features])
    
    # Transform the test data
    test_scaled = scaler.transform(test[numerical_features])
    
    # Convert the scaled arrays back to DataFrames
    train_scaled_df = pd.DataFrame(train_scaled, columns=numerical_features)
    test_scaled_df = pd.DataFrame(test_scaled, columns=numerical_features)
    
    # Replace the original columns with the standardized columns
    train[numerical_features] = train_scaled_df
    test[numerical_features] = test_scaled_df
    
    return train, test

In [7]:
# Data Processing 
train_data = noise_removal(train_data)
test_data = noise_removal(test_data)
train_data = missing_value_imputation(train_data)
test_data = missing_value_imputation(test_data)
train_data, test_data = feature_engineering(train_data, test_data)

There are 6 interaction features:
['Episode_Length_minutes_Host_Popularity_percentage', 'Episode_Length_minutes_Guest_Popularity_percentage', 'Episode_Length_minutes_Number_of_Ads', 'Host_Popularity_percentage_Guest_Popularity_percentage', 'Host_Popularity_percentage_Number_of_Ads', 'Guest_Popularity_percentage_Number_of_Ads']


In [ ]:
train_data, test_data = one_hot_encode_features(train_data, test_data, categorial_valiable)

#train_data, test_data = standardize_features(train_data, test_data, numerical_variables)

In [ ]:
X = train_data.drop(['Listening_Time_minutes', 'id'], axis=1)
y = train_data['Listening_Time_minutes']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(y_train), len(y_val))

In [ ]:
best_param_decision_tree = {'max_depth': 9, 'min_samples_split': 8, 'min_samples_leaf': 9}
best_param_bagging_reg = {"base_estimator" : DecisionTreeRegressor(max_depth=8, random_state=42), "n_estimators": 300}
best_param_rf = {"max_depth" : 9, "min_samples_split" : 8, "min_samples_leaf" : 9, "n_estimators" : 300}
best_param_xg = { "booster": "dart",
    "lambda": 9.770320148460564e-08,
    "alpha": 1.612824838455072e-05,
    "subsample": 0.8609881565231172,
    "colsample_bytree": 0.8432633524521188,
    "max_depth": 8,
    "min_child_weight": 6,
    "eta": 0.42989406584229434,
    "gamma": 0.000734224353473736,
    "grow_policy": "lossguide",
    "num_boost_round": 2756}
best_param_lgb = {'booster': 'dart',
     'lambda': 1.0688566030752065e-08,
     'alpha': 2.0008470252864347e-07,
     'subsample': 0.7258455740008215,
     'colsample_bytree': 0.9669207998867171,
     'max_depth': 8,
     'min_child_weight': 2,
     'eta': 0.41816603896355214,
     'gamma': 0.055128970324283245,
     'grow_policy': 'lossguide',
     'num_boost_round': 872,
     'device': 'gpu',
     'verbose': -1}

In [ ]:
# BaggingRegressor = 13.24550
best_bagging_model = BaggingRegressor(**best_param_bagging_reg, random_state=42, n_jobs=-1, verbose=10)
# RandomForestRegressor = 13.21520
best_rf_model = RandomForestRegressor(**best_param_rf, random_state=42, n_jobs=-1, verbose=10)
# DecisionTreeRegressor = 13.25657
best_tree_model = DecisionTreeRegressor(random_state=42, **best_param_decision_tree)
# XGBRegressor : 13.21074
best_param_xg["tree_method"] = "gpu_hist" #ensure GPU
best_param_xg["gpu_id"] = 0
best_param_xg["predictor"] = "gpu_predictor"
best_xgb_model = XGBRegressor(**best_param_xg)
# LGBMRegressor = 13.08369
best_param_lgb["device"] = "gpu" #ensure GPU
best_param_lgb['verbose'] = -1
best_lgbm_model = LGBMRegressor(**best_param_lgb)

In [ ]:
# Create a VotingClassifier with the best estimators
voting_clf = VotingRegressor(
    estimators=[
        ('tree', best_tree_model),
        ('xg', best_xgb_model),
        ('lgb', best_lgbm_model),
        ('rf', best_rf_model),
        ('bag', best_bagging_model)
    ],
    verbose=100
)

# Fit the VotingClassifier on your resampled training data
#voting_clf.fit(X_train, y_train)

In [13]:
voting_clf.fit(X, y)

building tree 9 of 300


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   41.3s


building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.0min


building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300


...

[Voting] ....................... (4 of 5) Processing rf, total=26.4min


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed: 25.1min
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed: 25.1min remaining: 25.1min


[Voting] ...................... (5 of 5) Processing bag, total=25.2min


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 25.2min finished


VotingRegressor(estimators=[('tree',
                             DecisionTreeRegressor(max_depth=9,
                                                   min_samples_leaf=9,
                                                   min_samples_split=8,
                                                   random_state=42)),
                            ('xg',
                             XGBRegressor(alpha=1.612824838455072e-05,
                                          base_score=None, booster='dart',
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.8432633524521188,
                                          device=None,
                                          early_stopping_rounds=None,
                                          enable_cat...
                                           num_boost_round=872,
                                           subsample=0.7258455740008215,
                                           verbose=-1)),
                            ('rf',
                             RandomForestRegressor(max_depth=9,
                                                   min_samples_leaf=9,
                                                   min_samples_split=8,
                                                   n_estimators=300, n_jobs=-1,
                                                   random_state=42,
                                                   verbose=10)),
                            ('bag',
                             BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=8,
                                                                                   random_state=42),
                                              n_estimators=300, n_jobs=-1,
                                              random_state=42, verbose=10))],
                verbose=100)

In [14]:
#y_val_pred = voting_clf.predict(X_val)
#print('DecisionTreeRegressor validation Room mean square error:', np.sqrt(mean_squared_error(y_val, y_val_pred)))

In [15]:
X_test = test_data.drop(['id'], axis=1)
test_predictions = voting_clf.predict(X_test)

submission_df = pd.DataFrame({
    'id': test_data['id'],
    'Listening_Time_minutes': test_predictions  # Predicted probabilities for rainfall
})

submission_df["Listening_Time_minutes"] = submission_df["Listening_Time_minutes"].clip(
    lower=0, upper=test_data["Episode_Length_minutes"])

submission_df.to_csv("submission_VotingRegressor_04_05_v2.csv", index=False)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:    1.4s
[Para